# Importar Librerías

In [ ]:
from pprint import pprint as print
import numpy as np
import pandas as pd
import pandas_profiling as pp
import datetime
import random
import catboost as ctb
from matplotlib import pyplot as plt
from mpl_toolkits.basemap import Basemap
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import classification_report

## Config librerías

In [ ]:
# Seed numpy and random to reproduce same results
np.random.seed(42)
random.seed(42)
# Pandas use plotly instead of matplotlib as backend
pd.options.plotting.backend = "plotly" 
# Pandas dataframe display config
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)

# Carga de Datos

## Carga datos desafío

In [ ]:
flights_raw = pd.read_csv('./dataset_SCL.csv')
flights = flights_raw.copy()
flights.head()

## Carga datos códigos aerolineas
**Fuente**: https://en.wikipedia.org/wiki/List_of_airline_codes

**Modificaciones**:
- Se agregaron un par de filas manualmente
- Se eliminaron duplicados por ICAO manualmente en el archivo

In [ ]:
# read csv
airline_codes = pd.read_csv('./airline_codes.csv')
airline_codes = airline_codes[['ICAO', 'Airline', 'Country/Region']] 
airline_codes.columns = [c.lower().replace('/','_') for c in airline_codes.columns]
airline_codes = airline_codes.drop_duplicates()
airline_codes.head()

## Carga datos códigos aeropuertos
**Fuente**: https://github.com/datasets/airport-codes

**Modificaciones**:
- Se agregaron un par de filas manualmente

In [ ]:
# read csv
airport_codes = pd.read_csv('./airport-codes.csv')
# create column for lat and long
airport_codes[['lat', 'lon']] = pd.DataFrame(airport_codes.coordinates.str.split(',').to_list()).astype(float)
# Dejamos solo las columnas de interés
airport_codes = airport_codes[['ident', 'type', 'name', 'lat', 'lon']]
airport_codes[['lat', 'lon']] = airport_codes[['lat', 'lon']].astype(float)
airline_codes = airline_codes.drop_duplicates()
airport_codes.head()


- `Des-I `: Código de ciudad de destino programado.
- `Des-O `: Código de ciudad de destino de operación.
- `Emp-I `: Código aerolínea de vuelo programado.
- `Emp-O `: Código aerolínea de vuelo operado.
- `Fecha-I` : Fecha y hora programada del vuelo.
- `Fecha-O `: Fecha y hora de operación del vuelo.
- `Ori-I `: Código de ciudad de origen programado.
- `Ori-O `: Código de ciudad de origen de operación
- `Vlo-I `: Número de vuelo programado.
- `Vlo-O `: Número de vuelo de operación del vuelo.
- `OPERA `: Nombre de aerolínea que opera.
- `SIGLADES `: Nombre ciudad destino.
- `SIGLAORI `: Nombre ciudad origen.
- `TIPOVUELO `: Tipo de vuelo, I =Internacional, N =Nacional.
- `AÑO `: Año de operación del vuelo.
- `MES `: Número de mes de operación del vuelo.
- `DIA `: Día del mes de operación del vuelo.
- `DIANOM `: Día de la semana de operación del vuelo.

## Formateo rápido de los datos

### Cambiamos los `-` por `_` y dejamos todo en minuscula en los nombres de columna por conveniencia y orden

In [ ]:
flights.columns = [c.replace('-', '_').lower() for c in flights.columns]

### Ordenamos las columnas para facilitar la lectura

In [ ]:
flights = flights[[c for c in flights.columns if '_' not in c] + sorted(list(flights.filter(regex='.*_.*').columns))]

### Asignamos los tipos de datos correctos a algunas columnas que se leyeron mal

In [ ]:
flights = flights.astype({'vlo_i':'str', 'vlo_o':'str', 'fecha_o':'datetime64[ns]', 'fecha_i':'datetime64[ns]'})

# Análisis exploratorio de los datos

## Utilizamos la librería `pandas-profiling` para tener una idea general de los datos 

In [ ]:
# generate report to get quick insights of the data
pd.options.plotting.backend = "matplotlib" 
profile = pp.ProfileReport(flights, title="Flights Report")
display(profile)
pd.options.plotting.backend = "plotly" 

In [ ]:
flights.info()

## Revisión de duplicados

In [ ]:
print(flights_raw.shape)
print(flights_raw.drop_duplicates().shape)

- No hay duplicados de filas enteras en los datos crudos

## Creamos columna con llave para identificar registros únicoas

In [ ]:
key_cols = ['vlo_i', 'vlo_o', 'emp_i', 'emp_o', 'fecha_i', 'fecha_o']
assert flights[key_cols].drop_duplicates().shape[0] == flights.shape[0]


In [ ]:
flights['pk'] = flights[key_cols].astype(str).agg('.'.join, axis=1).str.replace('\s', '+').str.replace(':','')

## Agregamos data de aeropuertos a vuelos

In [ ]:
s_original = flights.shape[0]
flights = flights.merge(airport_codes, left_on='des_i', right_on='ident', how='left')
flights = flights.merge(airport_codes, left_on='des_o', right_on='ident', how='left', suffixes=['_i', '_o'])
assert flights.ident_i.isna().sum() ==0, 'Airport data missing, add to csv file'
assert flights.ident_o.isna().sum() ==0, 'Airport data missing, add to csv file'
assert flights.shape[0] == s_original, 'Found duplicated airport ident, check csv file'

## Agregamos data de aerolineas a  vuelos

In [ ]:
s_original = flights.shape[0]
flights = flights.merge(airline_codes.rename(columns={'airline':'airline_i', 'country_region':'airline_countryreg_i'}), how='left', left_on='emp_i', right_on='icao')
flights = flights.merge(airline_codes.rename(columns={'airline':'airline_o', 'country_region':'airline_countryreg_o'}), how='left', left_on='emp_o', right_on='icao', suffixes=['_i', '_o'])
assert flights.icao_i.isna().sum() == 0, 'Airline data missing, add to csv file'
assert flights.icao_i.isna().sum() == 0, 'Airline data missing, add to csv file'
assert flights.shape[0] == s_original, 'Found duplicated airline ident, check csv file'

## Ploteamos la ubicación de los aeropuertos de destino programados vs de la operación
- Plot interactivo, hacer click en leyenda para activar desactivar puntos

In [ ]:
# Count number of times a destiny was programed
flights['des_name_i'] = flights.name_i + ' ('+ flights.des_i +')'
flights['des_name_o'] = flights.name_o + ' ('+ flights.des_o +')'
flights_des_i = flights.groupby(['des_name_i', 'lat_i', 'lon_i']).agg({'vlo_i':'count'}).rename(columns={'vlo_i':'n_flights'}).reset_index()
flights_des_i['source']='itinerary'
# Count number of times a destiny was flown to
flights_des_o = flights.groupby(['des_name_o', 'lat_o', 'lon_o']).agg({'vlo_o':'count'}).rename(columns={'vlo_o':'n_flights'}).reset_index()
flights_des_o['source']='operation'
flights.airline_i  = flights.airline_i + ' ('+ flights.emp_i +')'
flights.airline_o  = flights.airline_o + ' ('+ flights.emp_o +')'

In [ ]:
# Plot in map
flights_des_i.columns = flights_des_i.columns.str.rstrip('_i')
flights_des_o.columns = flights_des_o.columns.str.rstrip('_o')
flighs_des = pd.concat([flights_des_i, flights_des_o], axis=0)
fig = px.scatter_geo(flighs_des,lat='lat', lon='lon', projection="natural earth", hover_name="des_name",
                     color='source', size='n_flights', height=600, color_discrete_sequence=['#636EFA', '#FF7F0E'])
fig.show()

- El grueso de los vuelos se encuentra en sudamerica
- Pareciera ser que no hay mayores cambios entre los destinos de los vuelos programados y lo que sucede en la operación

## Ploteo ciudades de destino

In [ ]:
flights_by_des_o = flights.siglades.value_counts().reset_index().rename(columns={'index':'siglades', 'siglades':'n'})
flights_by_des_o['perc'] = flights_by_des_o.n/flights_by_des_o.n.sum()*100
fig = flights_by_des_o.plot.bar(x='siglades', y ='perc', height=350, text='n')
fig.update_xaxes(tickangle=42)
fig.update_layout(xaxis_title="Destino", yaxis_title="Vuelos (%)")

In [ ]:
flights_by_des_o['perc_cumsum'] = flights_by_des_o.perc.cumsum()
flights_by_des_o[:8][['siglades', 'perc', 'perc_cumsum' ]]

- 16 destinos acumulan el 80% de los vuelos
- 8 destinos acumulan el grueso de los vuelos (56%)

## Revisión vuelos que cambiaron de destino

In [ ]:
changed_flights_filter = flights.des_i != flights.des_o
print(f'Des-I same as Des-O: {(~changed_flights_filter).sum():,} ({(~changed_flights_filter).sum()/flights.shape[0]*100:.2f}%)')
print(f'N flights changed: {changed_flights_filter.sum()} ({changed_flights_filter.sum()/changed_flights_filter.shape[0]*100:.2f}%)')
flights[changed_flights_filter][['des_i','name_i', 'des_o', 'name_o', 'siglades']]

- Son pocos vuelos con cambio de destino y en general nacionales, probablemente emergencias

## Revisamos vuelos que cambiaron de aerolinea

In [ ]:
flights.airline_i  = flights.airline_i + ' ('+ flights.emp_i +')'
flights.airline_o  = flights.airline_o + ' ('+ flights.emp_o +')'
fig_i = flights.airline_i.value_counts()
fig_o = flights.airline_o.value_counts()
fig = pd.concat([fig_i, fig_o], axis=1).plot.bar(barmode='overlay', color_discrete_sequence=['#636EFA', '#FF7F0E'], opacity=.8, height=600)
fig.update_xaxes(tickangle=42)
fig.update_layout(title={'text': "Distribución aerolineas"}, legend=dict(yanchor="top", xanchor="right", x = .95))

- Latam chile pareciera delegar varios vuelos a Latam Express, quizá se programa todo con LAN pero en la operación se reasigna o puede ser que haya un problema de asignación desde un principio

## Revisamos que aerolineas programadas no aparecen en las de la operación y viceversa

In [ ]:
not_in_op = set(flights.emp_i.unique()).difference(flights.emp_o.unique())
not_in_op_flights = flights[flights.emp_i.isin(not_in_op)]
print(f'Airlines missing in operation: {not_in_op} ({not_in_op_flights.shape[0]} rows)')
print(f'Related operators: {not_in_op_flights.opera.unique()}')

not_in_it = set(flights.emp_o.unique()).difference(flights.emp_i.unique())
not_in_it_flights = flights[flights.emp_o.isin(not_in_it)]
print(f'Airlines missing in itinerary: {not_in_it} ({not_in_it_flights.shape[0]} rows)')
print(f'Related itinerary: {not_in_it_flights.opera.unique()}')

- Ver por que hay aerolineas que nunca aparecen en la operacion 
- 48O y 56R son de LAW, ya no existe esta aerolinea y pareciera ser un poerador virtual com se verá más abajo

### **Revisión QFU y QFA**

In [ ]:
display(flights[flights.emp_i=='QFU'][['emp_i', 'siglades']].siglades.unique())
display(flights[flights.emp_i=='QFU'].vlo_i.unique())
display(flights[flights.emp_o=='QFA'][['emp_o', 'siglades']].siglades.unique())
display(flights[flights.emp_o=='QFA'].vlo_o.unique())

In [ ]:
display(flights[(flights.emp_i=='QFU')&(flights.siglades=='Melbourne')])
display(flights[(flights.emp_o=='QFA')&(flights.siglades=='Melbourne')])

- `QFU` y `QFA` son ambas `Qantas` pero con código distinto, puede ser una fusión de empresas
- Solo un registro con destino `Melbourne` para el mismo número de vuelo, 

## Cantidad de vuelos por fecha, operador, destino aerolinea


In [ ]:
flights['date_i'] = flights.fecha_i.dt.date
flights['date_o'] = flights.fecha_i.dt.date

In [ ]:
print(f' date_i (n={flights.date_i.nunique()}) '.center(50,'='))
print(f'median:{flights.date_i.value_counts().median()}')
display(flights.date_i.value_counts().describe()[1:-1])
print(f' date_o (n={flights.date_o.nunique()}) '.center(50,'='))
print(f'median:{flights.date_o.value_counts().median()}')
display(flights.date_o.value_counts().describe()[1:-1])
print(f' emp_i (n={flights.emp_i.nunique()}) '.center(50,'='))
print(f'median:{flights.emp_i.value_counts().median()}')
display(flights.emp_i.value_counts().describe()[1:-1])
print(f' emp_o (n={flights.emp_o.nunique()}) '.center(50,'='))
print(f'median:{flights.emp_o.value_counts().median()}')
display(flights.emp_o.value_counts().describe()[1:-1])
print(f' siglades (n={flights.emp_i.nunique()}) '.center(50,'='))
print(f'median:{flights.siglades.value_counts().median()}')
display(flights.siglades.value_counts().describe()[1:-1])
print(f' opera (n={flights.opera.nunique()}) '.center(50,'='))
print(f'median:{flights.opera.value_counts().median()}')
display(flights.opera.value_counts().describe()[1:-1])

## Cantidad de vuelos por operador

In [ ]:
flights_by_operator = flights.opera.value_counts().reset_index().rename(columns={'index':'opera', 'opera':'n'})
flights_by_operator['perc'] = flights_by_operator.n/flights_by_operator.n.sum()*100
fig = flights_by_operator.plot.bar(x='opera', y ='perc', width=800, height=350, text='n')
fig.update_xaxes(tickangle=42)
fig.update_layout(xaxis_title="operador", yaxis_title="vuelos (%)")

- El operador más grande es `LATAM` y `Sky` por mucho (60% + 20% del total)

### Revisar % de vuelos que cambian de aerolinea por operador

In [ ]:
flights_by_operator = flights.opera.value_counts().reset_index().rename(columns={'index':'opera','opera':'n_all'})
changed_airline = flights.query('emp_i != emp_o').opera.value_counts().reset_index().rename(columns={'index':'opera','opera':'n'})
changed_airline = changed_airline.merge(flights_by_operator, how='left', on='opera')
changed_airline['perc'] = changed_airline.n/changed_airline.n_all*100
changed_airline.plot.bar(x='opera', y=['perc'], width=600, height=300, text='n').update_layout(xaxis_title="Operador", yaxis_title="Cambios en aerolinea original (%)")

In [ ]:
changed_airline.n.sum()/changed_airline.n_all.sum()*100

- `LATAM` tiene muchos cambios entre la aerolinea original del itinerario y la de la operación
- `Latin American Wings` y `Qantas` parece ser operadores virtuales, casi el 100% de sus vuelos cambian de aerolinea
- 40% de los vuelos cambia de aerolinea con respecto a la programada

## Horarios de viaje preferidos

In [ ]:
%matplotlib inline
fligths_time_vc = flights.fecha_i.dt.round('15min').dt.time.value_counts()
fligths_time_vc = fligths_time_vc.reset_index().rename(columns={'index':'fecha_i', 'fecha_i':'n_flights'})
fligths_time_vc['500>'] = fligths_time_vc.n_flights<700
fig = fligths_time_vc.sort_values(by='fecha_i').plot.bar(x='fecha_i', y='n_flights', title='Distribución de vuelos según hora', text='n_flights', color='500>', color_discrete_sequence=['#FF7F0E', '#636EFA'])
fig.update_layout(xaxis_title="Horario (intervalos de 15 min)", yaxis_title="N vuelos", xaxis_categoryorder = 'category ascending')

- Horario más ocupado parece ser entre `6:00` y `9:30`
- Baja en cantidad de `9:45` a `10:45`
- Horario entre `23:00` y `5:45` en general es bajo en cantidad de vuelos

## Comparamos la cantidad de vuelos entre dfechas programadas y las de la operación
- Eliminamos los vuelos que llegaron el 2018-01-01 solo para efectos de la visualización

In [ ]:
# Compute n_flights for fecha_o
fig_o = flights.fecha_o.dt.date.value_counts().sort_index()[:-1]
# Compute n_flights for fecha_i
fig_i = flights.fecha_i.dt.date.value_counts().sort_index()
# Plot lines
fig = pd.concat([fig_i, fig_o], axis=1).plot.line(color_discrete_sequence=['#636EFA', '#FF7F0E'], height=400, markers=True)
date_intervals_limits = pd.to_datetime(['2017-01-01', '2017-03-03', '2017-07-15', '2017-07-31', '2017-09-11', '2017-09-30', '2017-12-15', '2017-12-31'])
date_intervals_limits = date_intervals_limits.to_numpy().reshape((4,2))
for i, [ds,de] in enumerate(date_intervals_limits):
    fig.add_vrect(x0=str(ds), x1=str(de), 
                  annotation_text="Temporada Alta", annotation_position="top left",
                  fillcolor="#00CC96", opacity=0.25, line_width=0)
    
fig.add_vrect(x0='2017-03-04', x1='2017-07-02', 
              fillcolor="red", opacity=0.08, line_width=0)
fig.update_layout(xaxis={'tickangle':42}, title={'text': "Distribución aerolineas"}, legend=dict(yanchor="top", xanchor="right", x = .99, y=.3))

- Todos los sábados coincide que hay una caida en la cantidad de vuelos, podría ser por caida en la demanda o algún tema logístico del aeropuerto
- Fuerte alza en julio
- la cantidad de vuelos es muy estacionaria
- Baja a partir de fines de febrero (comienzan los colegios y termina la epoca de vacaciones) hasta inicios de julio [03-01/07-01]
- La definicion de temporada alta parece estar subdimencionada para julio o podría ser un caso puntual para 2017
- Las fechas con mayor cantidad de vuelo parecen ser de [01-01/03-01, 07-03/08-11, 11-13/12-12]

#### Insights
- Vuelos aumentan bastante entre junio y julio, luego en octubre hasta diciembre hay alza
- A fin de mes baja un poco la cantuidad de vuelos
- El hay mas vuelos nacionales que nacionales
- Los operadores principales son LAN y SKY por mucho
- Columnas `SIGLAORI` y `Ori-O` no entregan información, todos los vuelos salen de santiago
- Los vuelos de lan cambian mucho de aerolinea entre lan y lan express, ~30%, puede ser por que se creó lan express después que se hayan programado los viajes, o sino pueden haber problemas de asignación

## Cantidad de vuelos con más de 24 hr de atraso

In [ ]:
((flights.date_o - flights.date_i).astype(int).abs()>1).sum()

Vuelos no se atrasan más de un día

## Tipo de vuelo por operador

In [ ]:
flights.groupby(['opera', 'tipovuelo']).size().sort_values(ascending=False).reset_index().plot.bar(x='opera', y=0, color='tipovuelo', barmode='group')

In [ ]:
flights[flights.tipovuelo == 'N'].opera.unique()

- Muy pocos operadores que realicen vuelos nacionales
- El fuerte de SKY son vuelos nacionales

## Tipo de vuelo según días de la semana/meses

In [ ]:
flights.groupby(['dianom', 'tipovuelo']).size().sort_values(ascending=False).reset_index().plot.bar(x='dianom', y=0, color='tipovuelo', barmode='group', width=800, height=350, text=0)
#.bar(labels={'index':"Dia semana", 'value':"N vuelos", 'variable':''}, width=500, height=300, text='value')

- La baja en vuelos que se ve el sábado se debe principlamente a una caida en vuelos Nacionales, quizá es un tema de precios o que la gente viaja los viernes en la noche para aprovechar el fin de semana
- Internacionales se mantienen relativamente constantes con exepción del martes que bajan un poco

In [ ]:
dianom_by_opera = flights.groupby(['dianom', 'opera']).size().sort_values(ascending=False).reset_index()
n_flights_by_operator = dianom_by_opera.groupby('opera')[0].sum().reset_index().rename(columns={0:'n_all'})
dianom_by_opera = dianom_by_opera.merge(n_flights_by_operator, on='opera', how='left')
dianom_by_opera['perc'] = dianom_by_opera[0]/dianom_by_opera.n_all
dianom_by_opera.plot.bar(x='opera', y='perc', color='dianom', barmode='group', height=500, text=0)

In [ ]:
flights.groupby(['mes', 'tipovuelo']).size().reset_index().sort_values(by='mes').plot.line(x='mes', y=0, color='tipovuelo', width=800, height=350)

- Vuelos nacionales e internacionales se comportan parecido durante hasta septiembre dónde los vuelos nacionales se disparan

In [ ]:
flights['time_15'] = flights.fecha_i.dt.round('15min').dt.time
flights.groupby(['dianom', 'time_15']).size().sort_index().reset_index().plot.line(x='time_15', color ='dianom', y=0, height=350)

- No parece hae

# Generación `Synthetic Features`

## Temporada alta

In [ ]:
flights['fecha_i_doy'] = flights.fecha_i.dt.day_of_year
flights['temporada_alta'] = False
date_intervals_limits = pd.to_datetime(['2017-01-01', '2017-03-03', '2017-07-15', '2017-07-31', '2017-09-11', '2017-09-30', '2017-12-15', '2017-12-31']).day_of_year
date_intervals_limits = date_intervals_limits.to_numpy().reshape((4,2))
for [ds, de] in date_intervals_limits:
    flights['temporada_alta']  = flights['temporada_alta'] | ((flights['fecha_i_doy'] >= ds) & (flights['fecha_i_doy'] <= de))
flights['temporada_alta'] = flights['temporada_alta'].astype(int)
flights[['temporada_alta', 'fecha_i']].sample(10)

### Revisión promedio vuelos por día según temporada

In [ ]:
flights_by_season = flights.groupby('temporada_alta').agg({'date_i':'nunique', 'date_o':'nunique', 'dia':'count'}).rename(columns={'dia':'n'})
flights_by_season['mean_daily_flights_i'] =  flights_by_season.n/flights_by_season.date_i
flights_by_season['mean_daily_flights_o'] =  flights_by_season.n/flights_by_season.date_o
flights_by_season

- Las fechas de temporada alta parecen no impactar mucho en la cantidad de vuelos diarios, el promedio de vuelos diarios no varía mucho (+5%)
- Puede ser que el 2017 las fechas de temporada alta hayan cambiado c/r a años anteriores, o las fechas de temporada alta no están muy bien definidas

## Periodo día

In [ ]:
flights['fecha_i_time'] = flights.fecha_i.dt.time
# morning 5:00 - 11:59
morning = 1*((flights['fecha_i_time']>=datetime.time(5,0)) & (flights['fecha_i_time']<=datetime.time(11,59)))
# afternoon 12:00 - 18:59
afternoon = 2*((flights['fecha_i_time']>=datetime.time(12,0)) & (flights['fecha_i_time']<=datetime.time(18,59)))
# night 19:00 - 4:59
night =3*((flights['fecha_i_time']<=datetime.time(4,59)) | (flights['fecha_i_time']>=datetime.time(19,0)))
# Join conditions and replace number with category name
flights['periodo_dia'] = (morning + afternoon + night).map({1:'mañana', 2:'tarde', 3:'noche'})
flights[['periodo_dia', 'fecha_i_time']].sample(5)

In [ ]:
flights.periodo_dia.value_counts()

In [ ]:
flights.groupby(['siglades', 'periodo_dia']).size().reset_index().plot.bar(x='siglades', y=0, color='periodo_dia', barmode='group')

In [ ]:
flights.groupby(['opera', 'periodo_dia']).size().reset_index().plot.bar(x='opera', y=0, color='periodo_dia', barmode='group')

In [ ]:
flights.groupby(['tipovuelo', 'periodo_dia']).size().reset_index().plot.bar(x='tipovuelo', y=0, color='periodo_dia', barmode='group', width=500, height=300)

- Horario nocturno es el menos preferido para vuelos nacionales

## Diferencia en minutos entre hora planificada y hora real (min atraso)

In [ ]:
flights['dif_min'] = (flights.fecha_o - flights.fecha_i).astype('timedelta64[m]').astype(int)
flights[['fecha_i', 'fecha_o', 'dif_min']].sample(5)

In [ ]:
flights['dif_min'].value_counts().plot.bar()

In [ ]:
print(f'Kurtosis: {flights["dif_min"].kurt():.2f}')
print(f'Skewness: {flights["dif_min"].skew():.2f}')
flights["dif_min"].describe([.8, .82, .9, .95]).reset_index()

- Los tiempos de atraso parecen distribuir normal con alta kurtosis y un poco de asimetría
- ~82% tiene atraso >15 min

In [ ]:
flights_by_opera = flights.groupby('opera').agg({'dif_min':['count','mean']})
flights_by_opera.columns = ['n', 'mean']
flights_by_opera.reset_index().plot.bar(x='opera', y='mean', text='n')

In [ ]:
flights.plot.box(x='opera', y='dif_min', height=500)

In [ ]:
flights.plot.box(x='airline_i', y='dif_min', height=500)

In [ ]:
flights.plot.box(x='mes', y='dif_min', width=600, height=500)

In [ ]:
flights.plot.box(x='dianom', y='dif_min',width=600, height=500)

In [ ]:
flights.plot.box(x='tipovuelo', y='dif_min', width=400, height=400)

In [ ]:
flights.plot.box(x='temporada_alta', y='dif_min', width=400, height=400)

In [ ]:
flights.groupby('siglades').dif_min.mean().plot.bar(text='value')

Hay aerolineas, destinos y operadores que presentan calramente más retrasos que otras, probablemente el promedio en retrasos de cada una de estas dimensiones sea clave para lograr buenas predicciones

## Atraso mayor a 15 minutos

In [ ]:
flights['atraso_15'] = (flights['dif_min'] > 15).astype(int)
flights[['dif_min','atraso_15']].sample(5)

In [ ]:
flights['atraso_15'].sum()/flights.shape[0]

In [ ]:
flights.plot.box(x=['airline_i'], y='dif_min')

In [ ]:
flights.plot.box(x='siglades', y='dif_min')

In [ ]:
flights.plot.box(x='opera', y='dif_min')

### Destino

In [ ]:
delays_by_destination = flights.groupby(['siglades']).agg({'atraso_15':'sum', 'dia':'count'}).rename(columns={'dia':'n'})
delays_by_destination['perc'] = delays_by_destination.atraso_15/delays_by_destination.n
delays_by_destination['500<'] = delays_by_destination.n>500
fig = delays_by_destination.sort_values(by='perc', ascending=False).plot.bar(y='perc', text='n', color='500<', color_discrete_sequence=['#636EFA', '#FF7F0E'])
fig.update_layout(xaxis_title="Destino", yaxis_title="Atrasos (%)", xaxis_categoryorder = 'total descending')

### Aerolínea

In [ ]:
delays_by_airline = flights.groupby(['airline_i']).agg({'atraso_15':'sum', 'dia':'count'}).rename(columns={'dia':'n'})
delays_by_airline['perc'] = delays_by_airline.atraso_15/delays_by_airline.n
delays_by_airline['500<'] = delays_by_airline.n>500
fig = delays_by_airline.sort_values(by='perc', ascending=False).plot.bar(y='perc', text='n', color='500<', color_discrete_sequence=['#636EFA', '#FF7F0E'])
fig.update_layout(xaxis_title="Destino", yaxis_title="Atrasos (%)", xaxis_categoryorder = 'total descending')

In [ ]:
delays_by_airline = flights.groupby(['airline_o']).agg({'atraso_15':'sum', 'dia':'count'}).rename(columns={'dia':'n'})
delays_by_airline['perc'] = delays_by_airline.atraso_15/delays_by_airline.n
delays_by_airline['500<'] = delays_by_airline.n>500
fig = delays_by_airline.sort_values(by='perc', ascending=False).plot.bar(y='perc', text='n', color='500<', color_discrete_sequence=['#636EFA', '#FF7F0E'])
fig.update_layout(xaxis_title="Destino", yaxis_title="Atrasos (%)", xaxis_categoryorder = 'total descending')

In [ ]:
delays_by_destination = flights.groupby(['date_i']).agg({'atraso_15':'sum', 'dia':'count'}).rename(columns={'dia':'n'})
delays_by_destination['perc'] = delays_by_destination.atraso_15/delays_by_destination.n
delays_by_destination['500<'] = delays_by_destination.n>500
fig = delays_by_destination.sort_values(by='perc', ascending=False).plot.bar(y='perc', text='n', color='500<', color_discrete_sequence=['#636EFA', '#FF7F0E'])
fig.update_layout(xaxis_title="Destino", yaxis_title="Atrasos (%)", xaxis_categoryorder = 'total descending')

### Mes del año

### Día de la semana

### Tipo de vuelo

### Temporada

# Data Prep y Modelo

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

## Consideraciones

- Para predecir la probabilidad de vuelo no deberíamos tener nada de información de la operación por lo que no usaremos las variables asociadas a esta (`-O`), por ejemplo si sabemos que un vuelo cambió de destino con respecto a lo programado, o que cambió de aerolinea, número de vuelo u otros probablemente tenga una atraso, pero es algo que en la realidad no sabremos hasta que pase
- También hay que tener cuidado al armar las features para evitar `Data Leakage`, dependiendo de como armemos el set de test y train podría ocurrir que usamos información de test de forma inadvertida (al calcular promedios, porcentajes, u otros sobre toda la data en lugar de solo los datos de entrenamiento).
- Suponiendo que el  modelo debe ser capáz de predecir atrasos para cualqueir vuelo en cualquier epoca del año se hará un sampleo aleatorio de fechas para definir el set de train y test. Si tomamos los últimos X meses como el periodo de test el modelo queda ciego a cambios en el comportamiento que podrían darse debido a la epoca del año (sin tomar en cuenta la variable de temporada_alta que se construyó). Con esto queda descartado el uso de variables de lag que abarquen más de 1 día. Podrían haber sido utilles ya que si habían 10 vuelos programados para una fecha y salieron 9 
- Sería interesante extraer datos del clima del origen y el destino con una API ya qe afecta directamente a los vuelos y probablemente a los atrasos, pero por tiempo no se hizo 
- También podría agregarse datos de los vuelos del aeropuerto de destino, el vuelo podría salir perfecto pero si el aeropuerto de destino tiene problemas se atrasará igual
- Se descarta el uso de redes neuronales ya que no hay tantos muchos datos, podría hacers fine-tuning de un modelo pre-entrenado pero hay que buscarlo y modificarlo lo que es lento y no vale la pena en esta instancia
- Podrían usarse modelos de detección de anomalía pero como no hay muchas features del vuelo en sí se descartan
- Por tiempo se probaran solo modelos de arboles de decision, probablemente regresiones simples y modelos más tradicionales como SVM den peores resultados.
- La libreía de árboles a ser utilizada es CatBoost e XGBoost o LightGBM. Y hay variables categóricas que parecen ser importnates en este problema

## Creación set train y test

In [ ]:
# Create column to indicate day of year
flights['doy_i'] = flights.fecha_i.dt.day_of_year
# Pick randmo dates
dates_choice = np.random.choice(flights['doy_i'].unique(), size=int(flights['doy_i'].unique().shape[0]*.20), replace=False)
# Create index to split data into train and test
test_idx = flights['doy_i'].isin(dates_choice)
train_idx = ~test_idx

### Creación features generales
Creación de variables pára las que no necesitamos separar el set de test del set de train por problemas de data leakage

#### Codificamos variables ciclicas como la hora y el día del mes

In [ ]:
# Encode date of month
flights['dia_mes_cos'] = np.cos(2*np.pi*(flights['dia']/flights['dia'].max()))
flights['dia_semana_cos'] = np.cos(2*np.pi*(flights.fecha_i.dt.day_of_week/flights.fecha_i.dt.day_of_week.max()))
# Encode time of day
flights['time_i_num'] = flights.fecha_i.dt.round('5min').dt.time.astype(str).str.replace(':', '').astype(int)/100
flights['time_i_cos'] = np.cos(2*np.pi*(flights.time_i_num/flights.time_i_num.max()))

#### Agregamos algunas variables dummy

In [ ]:
# Add dummise for tipovuelo
flights = pd.concat([flights, pd.get_dummies(flights.tipovuelo, prefix='tv_')],axis=1)
# Add dummise for periodo_dia
flights = pd.concat([flights, pd.get_dummies(flights.periodo_dia, prefix='pd_')],axis=1)
# Add dummise for temporada_alta
flights = pd.concat([flights, pd.get_dummies(flights.temporada_alta, prefix='ta_')],axis=1)

#### Agregamos la cantidad de vuelos programados para el día

In [ ]:
# Add number of flights programmed for the day
flights = flights.merge(flights.groupby('date_i').dia.count().reset_index().rename(columns={'dia':'n_flights_date_i'}), on='date_i', how='left')

## Creación de splits de train y test

In [ ]:
# Set target column
target_col = 'atraso_15'
# Split train data and target column
train_x = flights[train_idx].drop(columns=target_col)
train_y = flights[train_idx][target_col]
# Split test data and target column
test_x = flights[test_idx].drop(columns=target_col)
test_y = flights[test_idx].atraso_15

## Creación de features post splits

### Agregamos distribución de vuelos por día de la seman para cada operador

In [ ]:
# Count flights by weekday for each operator
dianom_by_opera = train_x.groupby(['dianom', 'opera']).size().sort_values(ascending=False).reset_index()
n_train_x_by_operator = dianom_by_opera.groupby('opera')[0].sum().reset_index().rename(columns={0:'n'})
dianom_by_opera = dianom_by_opera.merge(n_train_x_by_operator, on='opera', how='left')
# Add percentage of flights by weekday and percentage of flights with respect to total
dianom_by_opera['perc'] = dianom_by_opera[0]/dianom_by_opera.n
dianom_by_opera['perc_total'] = dianom_by_opera.n/dianom_by_opera.n.sum()
dianom_by_opera_dist = dianom_by_opera.pivot_table(columns='dianom', values='perc', index='opera').fillna(0)
dianom_by_opera_dist = dianom_by_opera_dist.reset_index()

#### Calculamos promedio y desviación de tiempo de atraso del operador

In [ ]:
delay_by_opera = train_x.groupby('opera').agg({'dia':['mean','std']})
delay_by_opera.columns = ['opera_mean', 'opera_std']
delay_by_opera = delay_by_opera

#### Calculamos promedio y desviación de tiempo de atraso de la aerolinea programada

In [ ]:
delay_by_airline = train_x.groupby('emp_i').agg({'dia':['mean','std']})
delay_by_airline.columns = ['emp_mean', 'emp_i_std']
delay_by_airline = delay_by_airline

#### Calculamos promedio y desviación de tiempo de atraso del destino

In [ ]:
delay_by_dest = train_x.groupby('des_i').agg({'dia':['mean','std']})
delay_by_dest.columns = ['des_mean', 'des_i_std']
delay_by_dest = delay_by_dest

#### Agregamos las features de tiempod e atraso

In [ ]:
# Add operator weekdays distribution
train_x = train_x.merge(dianom_by_opera_dist, how='left', on='opera')
test_x = test_x.merge(dianom_by_opera_dist, how='left', on='opera')
# Add operator time diff distribution
train_x = train_x.merge(delay_by_opera, how='left', on='opera')
test_x = test_x.merge(delay_by_opera, how='left', on='opera')
# Add operator time diff distribution
train_x = train_x.merge(delay_by_airline, how='left', on='emp_i')
test_x = test_x.merge(delay_by_airline, how='left', on='emp_i')
# Add operator time diff distribution
train_x = train_x.merge(delay_by_dest, how='left', on='des_i')
test_x = test_x.merge(delay_by_dest, how='left', on='des_i')

#### Definimos las features a ser utilizadas

In [ ]:
# Set feature names
features = ['tipovuelo', 'siglades', 'opera', 'temporada_alta', 'periodo_dia', 'emp_i', 'dianom', 'dia_mes_cos', 'dia_semana_cos', 'time_i_cos', 'n_flights_date_i'] +\
list(dianom_by_opera_dist.drop(columns=['opera']).columns)+\
list(delay_by_dest.columns)+\
list(delay_by_airline.columns)+\
list(delay_by_opera.columns)+\
list(flights.filter(regex='tv_.*').columns) + \
list(flights.filter(regex='pd_.*').columns)+ \
list(flights.filter(regex='ta_.*').columns)
# Set categorical feature names
cat_features = ['tipovuelo', 'siglades', 'opera', 'temporada_alta', 'periodo_dia', 'emp_i', 'dianom']


#### Definimos train test pool

In [ ]:
pool_train = ctb.Pool(train_x[features], train_y,cat_features = cat_features)
pool_test = ctb.Pool(test_x[features], cat_features = cat_features)

#### Revisamos el desbalanceo de las clases para corregirlo en la misma librería del modelo

In [ ]:
weights = 1 - (train_y.value_counts()/train_y.shape[0]).values
weights

#### Elegimos los mejores parámetros

In [ ]:
## Initialize CatBoostRegressor
#model = ctb.CatBoostClassifier(class_weights=weights, logging_level='Silent')
#grid = {'iterations':[50, 100, 500],
#        'learning_rate': [0.03, 0.1],
#        'depth': [3, 5]}
#grid_search_result = model.grid_search(grid, pool_train)
#print(grid_search_result['params'])

#### Enrenamos el modelo

In [ ]:
# Fit model
#model = ctb.CatBoostClassifier(max_depth=5, loss_function='Logloss')
weights = {0:.2, 1:.8}
model = ctb.CatBoostClassifier(class_weights=weights, depth= 3, iterations= 500, learning_rate= 0.1, logging_level='Silent')
model.fit(pool_train)


#### Generamos las predicciones de enterenamiento y evaluamos

In [ ]:
# Get predictions
preds = model.predict(train_x[features])
preds_prob = model.predict_proba(train_x[features])

clf_report = classification_report(train_y, preds)
print(clf_report)
pd.DataFrame(preds)[0].describe()

In [ ]:
print(f'mean confidence is late:{preds_prob[train_y==1, 1].mean()}')
print(f'% is late:{(preds==1).sum()/preds.shape[0]*100}')

#### Revisamos que variables son las mas relevantes

In [ ]:
import shap
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(train_x[features])
# visualize the first prediction's explanation
shap.summary_plot(shap_values, train_x[features], max_display=30)

 - Variable n_flights_date_i parece tener un alto impacto en la predicción del modelo
 - El promedio de atraso de los destinos tambien parece ser informativca y util par el modelo
 - Falta probar distintas combianciones de variables y ver cuales funcionan mejor
 - a veces si una variable tiene mucho peso hace que el modleo no mire le resto, hay que ver como se comprota el modelo al quitar el número de vuelos por día que tiene mucho impaco

#### Evaluamos el desempeño en el set de test

In [ ]:
# Get predictions
preds = model.predict(test_x[features])
preds_prob = model.predict_proba(test_x[features])

clf_report = classification_report(test_y, preds)
print(clf_report)

In [ ]:
print(f'mean confidence is late:{preds_prob[test_y==1, 1].mean()}')
print(f'% is late:{(preds==1).sum()/preds.shape[0]*100}')

- Lamentablemente faltó tiempo para complementar con más datos, crear nuevas variables y poder hacer una buena selección de las mejores
- El desempeño del modelo aún puede mejorarse bastante, es muy poco preciso y sería ideal subir el recall de los atrasos para poder tomar acciones que permitan facilitar el manejo logístico en el aeropuerto con anticipación
- Dependiendo de si hay recursos se podría enfocar esfuerzos en mejorar la precisión del modleo para evitar falsas alarmas